# Coordinate descent

CuML's `lasso` and `elastic net` implementations are both able to use the coordinate descent solver. Lasso extends linear regression by providing L1 regularization and elastic net extends linear regression by providing a combination of L1 and L2 regularizers.

A tremendous speed up can be demonstrated for datasets with a large number of rows and fewer columns. Furthermore, the mean squared error (MSE) value for cuML's implementation is much smaller than the Scikit-learn implementation on very small datasets.

The model can take array-like objects, either in host as NumPy arrays or in device (as Numba or _cuda_array_interface_compliant), as well  as cuDF DataFrames. 

For information about cuDF, refer to the [cuDF documentation](https://rapidsai.github.io/projects/cudf/en/latest/) 

For information about cuML's lasso implementation, refer to the [cuML documentation](https://rapidsai.github.io/projects/cuml/en/latest/index.html)

For information about cuML's elastic net implementation, refer to the [cuML documentation](https://rapidsai.github.io/projects/cuml/en/latest/index.html)

In [2]:
import os

import numpy as np

import pandas as pd
import cudf as gd

from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error

from cuml.linear_model import Lasso as cuLasso
from sklearn.linear_model import Lasso as skLasso

from cuml.linear_model import ElasticNet as cuElasticNet
from sklearn.linear_model import ElasticNet as skElasticNet

ModuleNotFoundError: No module named 'cudf'

## Define Parameters

In [ ]:
n_samples = 2**21
n_features = 500

learning_rate = 0.001
algorithm = "cyclic"

## Generate Data

In [ ]:
%%time
X,y = make_regression(n_samples=n_samples, n_features=n_features,r andom_state=0)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=0)

In [ ]:
%%time
X_cudf = gd.DataFrame.from_pandas(X_train)
X_cudf_test = gd.DataFrame.from_pandas(X_test)

y_cudf = cudf.Series(y_train.values[:, 0])

## Lasso

### Scikit-learn Model

#### Fit

In [1]:
%%time
skols = Lasso(alpha=np.array([learning_rate]), 
              fit_intercept = True, 
              normalize = False,
              max_iter = 1000,
              selection=algorithm,
              tol=1e-10)

skols.fit(X_train, y_train)

NameError: name 'Lasso' is not defined

#### Predict

In [ ]:
%%time
sk_predict = skols.predict(X_test)

error_sk = mean_squared_error(y_test,sk_predict)

### cuML Model

#### Fit

In [ ]:
%%time
cuols = cuLasso(alpha=np.array([learning_rate]),
                fit_intercept = True,
                normalize = False,
                max_iter = 1000,
                selection=algorithm,
                tol=1e-10)

cuols.fit(X_cudf, y_cudf)

#### Predict

In [ ]:
%%time
# Calculate the mean squared error of the testing dataset using the cuml linear regression model
cu_predict = cuols.predict(X_cudf_test).to_array()
error_cu = mean_squared_error(y_test,cu_predict)

### Evaluate Results

In [ ]:
# Print the mean squared error of the sklearn and cuml model to compare the two
print("SKL MSE(y):")
print(error_sk)
print("CUML MSE(y):")
print(error_cu)

## Elastic Net

The elastic net model implemented in cuml contains the same parameters as the lasso model.
In addition to the variable values that can be altered in lasso, elastic net has another variable who's value can be changed: `l1_ratio` decides the ratio of amount of L1 and L2 regularization that would be applied to the model. When `l1_ratio = 0`, the model will have only L2 reqularization shall be applied to the model. (default = 0.5)

### Scikit-learn Model

#### Fit

In [ ]:
%%time
elastic_sk = ElasticNet(alpha=np.array([learning_rate]), 
                        fit_intercept = True, 
                        normalize = False, 
                        max_iter = 1000, 
                        selection=algorithm, 
                        tol=1e-10)

elastic_sk.fit(X_train, y_train)

#### Predict

In [ ]:
%%time
sk_predict_elas = elastic_sk.predict(X_test)

error_sk_elas = mean_squared_error(y_test,sk_predict_elas)

### CuML Model

#### Fit

In [ ]:
%%time
elastic_cu = cuElasticNet(alpha=np.array([learning_rate]), 
                          fit_intercept = True, 
                          normalize = False, 
                          max_iter = 1000, 
                          selection=algorithm, 
                          tol=1e-10)

elastic_cu.fit(X_cudf, y_cudf)

#### Predict

In [ ]:
%%time
cu_predict_elas = elastic_cu.predict(X_cudf_test).to_array()

error_cu_elas = mean_squared_error(y_test,cu_predict_elas)

### Evaluate Results

In [ ]:
print("SKL MSE(y):")
print(error_sk_elas)
print("CUML MSE(y):")
print(error_cu_elas)